# 一个简单的attention：将输入日期转换成机器可读日期

In [ ]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)#将输入重复TX次
concatenator = Concatenate(axis=-1)#特征融合层，相当于将数组按不同维度拼接
densor1 = Dense(10, activation = "tanh")#全连接层，10代表输出维度
densor2 = Dense(1, activation = "relu")#
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)#将张量进行点积


In [4]:
# GRADED FUNCTION: one_step_attention

def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.

    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)

    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """

    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s=repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    a=concatenator([s,a])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    ee=densor1(a)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    e=densor2(ee)#两个全连接层构成一个小神经网络，为了通过s和a预测得到e
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    al=activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context=doctor([al,a])
    ### END CODE HERE ###

    return context

In [6]:
n_a = 32
n_s = 64#s的维数，即上层LSTM的输出维数
post_activation_LSTM_cell = LSTM(n_s, return_state = True)#上层LSTM，有n_s个时间步，return_state参数，配置RNN层以返回列表，其中第一个是输出，下一个是内部RNN状态，用于恢复编码器的状态
output_layer = Dense(10000, activation=softmax)#输出层，全连接层

NameError: name 'softmax' is not defined

In [ ]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """

    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))#下层LSTM的输入层
    s0 = Input(shape=(n_s,), name='s0')#上层输入层的隐藏值
    c0 = Input(shape=(n_s,), name='c0')#上层输入层的激活值（相当于a0）
    s = s0
    c = c0

    # Initialize empty list of outputs
    outputs = []

    ### START CODE HERE ###

    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)#定义下层双向LSTM，其中Bidirectional代表双向RNN，LSTM参数中的return_sequences=True表示每个时间步都输出
    # Step 2: Iterate for Ty steps
    for t in range(Ty):

        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        con=one_step_attention(a,s)
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state = [s, c])#这一层返回了3个参数，输出、编码器状态，c
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        

        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
       

    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
   

    ### END CODE HERE ###

    return model
